# Busca em grafos 

In [1]:
import sys
  
# Insert the path of modules folder 
sys.path.insert(0, "C:\\Users\\Guilherme\\Documents\\Faculdade\\p5\\TG")
  
from grafo import *
import numpy as np

In [2]:
class Clock:
    def __init__(self):
        self.t = 0
    
    def __add__(self, v):
        self.t += v

In [3]:
class Node_Depth:
    def __init__(self, v, pai=None, PE=0, PS=0):
        self.v = v
        self.pai = pai
        self.PE = PE
        self.PS = PS

In [4]:
class busca_em_profundidade():

    def __init__(self, grafo):

        self.grafo = grafo
        self.nodes = [Node_Depth(v=v)  for v in range(1, len(grafo) + 1)]
        self.clock = Clock()
        self.matriz_de_cores = [[None] * len(self.grafo) for i in range(len(self.grafo))]
        
    def run(self, v):
        self.clock + 1
        self.nodes[v - 1].PE = self.clock.t
        
        for w in self.grafo.adjacency_list[v - 1]:
            
            if self.nodes[w - 1].PE == 0:
                if v < w:
                    self.matriz_de_cores[v - 1][w - 1] = "0,0,255" # azul para aresta de árvore
                else:
                    self.matriz_de_cores[w - 1][v - 1] = "0,0,255" # azul para aresta de árvore
                self.nodes[w - 1].pai = v
                self.run(w)
                
            elif w != self.nodes[v - 1].pai and self.nodes[w - 1].PS == 0:
                if v < w:
                    self.matriz_de_cores[v - 1][w - 1] = "255,0,0" # vermelho para aresta de retorno
                else:
                    self.matriz_de_cores[w - 1][v - 1] = "255,0,0" # vermelho para aresta de retorno
        
        self.clock + 1
        self.nodes[v - 1].PS = self.clock.t
    
    def gerar_gdf(self, name, path=''):                
        gdf_str = "nodedef>name VARCHAR,label VARCHAR\n"
        
        for i in range(1, len(self.grafo) + 1):
            gdf_str += f"{i},{i}\n"

        gdf_str += "edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN,color VARCHAR\n"
        
        for i in range(len(self.grafo)):
            for j in range(i + 1, len(self.grafo)):
                if self.matriz_de_cores[i][j] != None:
                    gdf_str += f"{i + 1},{j + 1},false,'{self.matriz_de_cores[i][j]}'\n"

        if path == '':
            with open(f"depth_solution_{name}.gdf", "w") as arquivo:
                arquivo.write(gdf_str)
        else:
            with open(path + '\\' + f"depth_solution_{name}.gdf", "w") as arquivo:
                arquivo.write(gdf_str)
        
grafo_name = "graph_17"
arquivo_graph = open(f"../instancias/{grafo_name}", 'r').readlines()
m, l = read_graph(arquivo_graph)
grafo = Graph(m, l)
algoritmo = busca_em_profundidade(grafo)
algoritmo.run(1)
algoritmo.gerar_gdf(grafo_name, ".\solucoes")

In [5]:
class Node_Breadth:
    def __init__(self, v, pai=None, nivel=0, L=0):
        self.v = v
        self.pai = pai
        self.nivel = nivel
        self.L = L

In [6]:
class busca_em_largura():

    def __init__(self, grafo):
        self.grafo = grafo
        self.nodes = [Node_Breadth(v=v)  for v in range(1, len(grafo) + 1)]
        self.clock = Clock()
        self.matriz_de_cores = [[None] * len(self.grafo) for i in range(len(self.grafo))]
        
    def run(self, V):
        
        self.nodes = [Node_Breadth(v=v)  for v in range(1, len(self.grafo) + 1)]
        self.clock = Clock()
        self.matriz_de_cores = [[None] * len(self.grafo) for i in range(len(self.grafo))]

        fila = [V]
        for node in self.nodes:
            if node.v == V:
                self.clock + 1
                node.L = self.clock.t
                
        while len(fila) > 0:
            
            v = fila.pop(0)
            node_v = self.nodes[v - 1]
            for w in self.grafo.adjacency_list[v - 1]:
                node_w = self.nodes[w - 1]
                
                if node_w.L == 0:
                    if v < w:
                        self.matriz_de_cores[v - 1][w - 1] = "0,0,255" # azul - pai
                    else:
                        self.matriz_de_cores[w - 1][v - 1] = "0,0,255" # azul - pai

                    node_w.pai = v
                    node_w.nivel = node_v.nivel + 1
                    self.clock + 1
                    node_w.L = self.clock.t
                    fila.append(w)
                    

                elif node_w.nivel == node_v.nivel:
                    if node_w.pai == node_v.pai:
                        if v < w:
                            self.matriz_de_cores[v - 1][w - 1] = "255,0,0" # vermelho - irmao
                        else:
                            self.matriz_de_cores[w - 1][v - 1] = "255,0,0" # vermelho - irmao
                    else:
                        if v < w:
                            self.matriz_de_cores[v - 1][w - 1] = "255,255,0" # amarelo - primo
                        else:
                            self.matriz_de_cores[w - 1][v - 1] = "255,255,0" # amarelo - primo

                elif node_w.nivel == node_v.nivel + 1: 
                    if v < w:
                        self.matriz_de_cores[v - 1][w - 1] = "0,255,0" # verde - tio
                    else:
                        self.matriz_de_cores[w - 1][v - 1] = "0,255,0"  # verde - tio


                
    
    def gerar_gdf(self, name, path=''):                
        gdf_str = "nodedef>name VARCHAR,label VARCHAR\n"
        for i in range(1, len(self.grafo) + 1):
            gdf_str += f"{i},{i}\n"
        gdf_str += "edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN,color VARCHAR\n"

        for i in range(len(self.grafo)):
            for j in range(i + 1, len(self.grafo)):
                if self.matriz_de_cores[i][j] != None:
                    gdf_str += f"{i + 1},{j + 1},false,'{self.matriz_de_cores[i][j]}'\n"
                    
        if path == '':
            with open(f"breadth_solution_{name}.gdf", "w") as arquivo:
                arquivo.write(gdf_str)
        else:
            with open(path + '\\' + f"breadth_solution_{name}.gdf", "w") as arquivo:
                arquivo.write(gdf_str)
    
    def exentricidade(self):
        exentricidades = []
        for v in range(1, len(self.grafo) + 1):
            self.run(v)
            max_nivel = self.nodes[0].nivel
            for i in range(1, len(self.nodes)):
                if max_nivel < self.nodes[i].nivel:
                    max_nivel = self.nodes[i].nivel
            exentricidades.append(max_nivel)
        raio, diametro = min(exentricidades), max(exentricidades)
        
        return raio, diametro
        
    def mean_distance(self):
        all_distances = [[None] * len(self.grafo) for i in range(len(self.grafo))]
        for v in range(1, len(self.grafo) + 1):
            self.run(v)
            for i, node in enumerate(self.nodes):
                all_distances[v - 1][i] = node.nivel
        
        all_distances = np.array(all_distances)

        soma = 0
        n = 0
        for i in range(len(self.grafo)):
            for j in range(i + 1, len(self.grafo)):
                soma += all_distances[i][j]
                n += 1
        
        return soma / n
    
    def relatorio(self):
        r, d = self.exentricidade()
        m = self.mean_distance()
        print(  f"\nRaio: {r}\n"
                f"Diametro: {d}\n"
                f"Media: {m}")

graph_name = "graph_20"

arquivo_graph_1 = open(f"../instancias/{graph_name}", 'r').readlines()
m1, l1 = read_graph(arquivo_graph_1)
grafo_1 = Graph(m1, l1)

algoritmo = busca_em_largura(grafo_1)
algoritmo.run(1)
algoritmo.gerar_gdf(graph_name, ".\solucoes")
algoritmo.relatorio()


Raio: 2
Diametro: 2
Media: 1.3853061224489795


In [7]:
def main():
    for i in range(1, 21):
        file_name = f"graph_{i}"
        arquivo_graph = open(f"../instancias/{file_name}", 'r').readlines()
        m, l = read_graph(arquivo_graph)
        grafo = Graph(m, l)
        
        algoritmo_depth = busca_em_profundidade(grafo)
        algoritmo_depth.run(1)
        algoritmo_depth.gerar_gdf(file_name, ".\solucoes")
        
        algoritmo_breadth = busca_em_largura(grafo)
        algoritmo_breadth.run(1)
        algoritmo_breadth.gerar_gdf(file_name, ".\solucoes")
        algoritmo_breadth.relatorio()


main()


Raio: 3
Diametro: 5
Media: 2.7111111111111112

Raio: 3
Diametro: 4
Media: 2.0444444444444443

Raio: 3
Diametro: 5
Media: 2.3777777777777778

Raio: 2
Diametro: 2
Media: 1.3777777777777778

Raio: 3
Diametro: 6
Media: 2.763157894736842

Raio: 2
Diametro: 3
Media: 1.768421052631579

Raio: 2
Diametro: 3
Media: 1.5421052631578946

Raio: 2
Diametro: 2
Media: 1.3842105263157896

Raio: 3
Diametro: 6
Media: 2.7655172413793103

Raio: 2
Diametro: 3
Media: 1.7724137931034483

Raio: 2
Diametro: 3
Media: 1.606896551724138

Raio: 2
Diametro: 2
Media: 1.393103448275862

Raio: 3
Diametro: 5
Media: 2.6166666666666667

Raio: 2
Diametro: 3
Media: 1.7076923076923076

Raio: 2
Diametro: 3
Media: 1.617948717948718

Raio: 2
Diametro: 2
Media: 1.3692307692307693

Raio: 3
Diametro: 5
Media: 2.4457142857142857

Raio: 2
Diametro: 3
Media: 1.7093877551020409

Raio: 2
Diametro: 2
Media: 1.6114285714285714

Raio: 2
Diametro: 2
Media: 1.3853061224489795
